# Notebook for Work with Optimization and Pool and Executor instances
For possible application in ModelFitting in brian2tools

Notebook goals: 
    - create instances with Scipy and Nevergrad that take two params and return optimal error
    - modify them to use the workers with Executor and Pool 
    - try using NeuronGrouop to adapt them

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from scipy.optimize._differentialevolution import DifferentialEvolutionSolver

from nevergrad.optimization import optimizerlib
from concurrent import futures
from nevergrad import instrumentation as inst

## Scipy

In [44]:
def test(x):
    arg2 = x[0] - x[1]
    return arg2

In [45]:
bounds = [(-5, 5), (-10, 0)]
result = differential_evolution(test, bounds)
result.x, result.fun

(array([-5.,  0.]), -5.0)

### use workers to map

In [4]:
with mp.Pool(2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [5]:
print(solver.x)
print(test(solver.x))

[-5.  5.]
-10.0


In [6]:
with DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=mp.Pool(2).map) as solver:
    solver.solve()

In [7]:
differential_evolution(test, bounds, updating='deferred', workers=mp.Pool(2).map)

     fun: -10.0
     jac: array([ 1.00000008, -1.00000008])
 message: 'Optimization terminated successfully.'
    nfev: 576
     nit: 18
 success: True
       x: array([-5.,  5.])

In [8]:
print(solver.x)
print(test(solver.x))

[-5.  5.]
-10.0


### Using Like for The Nevergrad PoolExecutor with Scipy optimize

In [9]:
with futures.ProcessPoolExecutor(max_workers=2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [10]:
print(solver.x)
print(test(solver.x))

[-5.  5.]
-10.0


### Nevergrad

In [72]:
# define the variables to be a bounded array
arg1 = inst.var.Array(2).bounded(a_min=-5, a_max=5)

instrum = inst.Instrumentation(arg1)
instrum

Instrumentation(Array(shape=(2,), transforms=[ArctanBound(a_max=5, a_min=-5)]))

In [12]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)

In [13]:
optim.optimize(test)

Candidate(args=(array([-5.,  5.]),), kwargs={}, data=[-3.97165157e+11  4.50108411e+10])

#### Verbosity
you can set up verbosity parameter to report on fitness of the function

In [27]:
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)

with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as executor:
    recommendation = optim.optimize(test, executor=executor, batch_mode=True, verbosity=0)

/home/alteska/anaconda3/envs/brian_opt/lib/python3.6/site-packages/nevergrad/common/tools.py:168: UserWarning: Ignoring since timer was stopped before starting.
  warnings.warn("Ignoring since timer was stopped before starting.")


In [15]:
recommendation

Candidate(args=(array([-4.99619639,  4.99608577]),), kwargs={}, data=[-836.86246351  813.21160721])

### Nevergrad with Executor

In [16]:
# can not be used the other way around, needs "submit" method

# optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)
# with mp.Pool(processes=optim.num_workers) as executor:
#     recommendation = optim.optimize(test, executor=executor, batch_mode=True)

### Two Separate Bounds Problem
Multiple bounds can not be passed for an array, but array can be cast as scalar and then used. 

In [75]:
# This kind of approaches won't work:
# a_max = {5, 5}
# a_min = {-5, -5}
# arg1 = inst.var.Array(2).bounded(a_min=a_min, a_max=a_max)

# a_max = [5, 5]
# a_min = [-5, -5]
# arg1 = inst.var.Array(2).bounded(a_min=a_min, a_max=a_max)

#### Two Variables Approach

In [85]:
def test2(x, y):
    arg2 = x - y
    return arg2

In [89]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

In [90]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)
optim.optimize(test2)

Candidate(args=(-4.999980489920258, 9.999905211467883), kwargs={}, data=[-163151.50443934   33581.05447804])

#### Args Approach

In [124]:
def test_args(*args):
    return args[0] - args[1]

In [125]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

In [126]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000)
optim.optimize(test_args)

Candidate(args=(-4.999999994745679, 9.999999976692884), kwargs={}, data=[-6.05805871e+08  1.36571971e+08])

### Test with More complicated function

In [103]:
def ackley(x):
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e

In [111]:
def ackley_args(*args):
    arg1 = -0.2 * np.sqrt(0.5 * (args[0] ** 2 + args[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * args[0]) + np.cos(2. * np.pi * args[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e

#### Scipy Verison of this optimization

In [112]:
bounds = [(-5, 5), (-10, 0)]

In [113]:
res = differential_evolution(ackley, bounds, updating='deferred', workers=mp.Pool(2).map, polish=False)
res

     fun: 4.440892098500626e-16
 message: 'Optimization terminated successfully.'
    nfev: 4170
     nit: 138
 success: True
       x: array([0., 0.])

#### Nevergrad

In [119]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=15000, num_workers=2)

In [120]:
with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as executor:
    recommendation = optim.optimize(ackley_args, executor=executor, batch_mode=True, verbosity=0)

In [121]:
recommendation

Candidate(args=(-8.520153765040456e-09, 0.9521665367008145), kwargs={}, data=[-2.67668525e-09 -3.24269587e+00])

## Using Instrumentation SoftmaxCategorical ( utils.Variable[X] ):
There are other ways to create variables, however they are not boundable

Discrete set of n values transformed to a n-dim continuous variable. Each of the dimension encodes a weight for a value, and the softmax of weights provide probabilities for each possible value. A random value is sampled from this distribution. Since the chosen value is drawn randomly, the use of this variable makes deterministic functions become stochastic, hence "adding noise"


Input: list
    a list of possible values for the variable

In [80]:
# in case of Array we can bound it's general output in following way
var = inst.var.Array(2, 2).bounded(3, 5, transform="arctan")
data = np.array([-10, 10, 0, 0])
output = var.data_to_argument(data)
output

array([[3.06345103, 4.93654897],
       [4.        , 4.        ]])

In [81]:
token = inst.var.SoftmaxCategorical(["blu", "blublu", "blublublu"], deterministic=True)
token.data_to_argument([1, 1, 1.01], deterministic=False)

'blublublu'

In [83]:
token = inst.var.SoftmaxCategorical(["blu", "blublu", "blublublu"])
token.data_to_argument([.5, 1, 2.])
token.data_to_argument(token.argument_to_data("blu"), deterministic=True)

'blu'

### Test Ask and Tell 

In [152]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

In [153]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000)

In [156]:
for _ in range(10):
    x = optim.ask()
    value = test_args(*x.args, **x.kwargs)
    optim.tell(x, value)

In [157]:
recommendation = optim.provide_recommendation()
recommendation

Candidate(args=(-4.911383245891773, 9.932189466048584), kwargs={}, data=[-35.91055094  46.93396634])

In [158]:
optim.optimize(test_args)

Candidate(args=(-4.926469413852488, 9.961740251062164), kwargs={}, data=[-43.28175274  83.19305952])

In [150]:
recommendation.args

(-4.986960917484704, 9.993636140880133)

In [151]:
recommendation.uuid

'2c9f1bb072f34feb9c8835d49b55dd00'